<a href="https://colab.research.google.com/github/WellcomePeujio/Optimizacion-de-Portafolios-con-el-Modelo-de-Markowitz/blob/main/Optimizaci%C3%B3n_de_Portafolios_con_el_Modelo_de_Markowitz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Optimización de Portafolios con el Modelo de Markowitz

**Problema**: Un portafolio está compuesto por tres activos con las siguientes características:

| Activo | Rendimiento Esperado (%) | Riesgo (%) | Correlación |
|--------|---------------------------|------------|-------------|
| A      | 10                        | 15         | 0.5         |
| B      | 8                         | 10         | 0.4         |
| C      | 12                        | 20         | 0.3         |

**Preguntas**:

1. Encuentra las proporciones óptimas para minimizar el riesgo del portafolio dado un rendimiento objetivo del 9%.
2. Genera un gráfico de la **frontera eficiente**.



# Resolución del Problema de Optimización de Portafolios con el Modelo de Markowitz

## Paso 1: Parámetros del Problema

Dado que el portafolio está compuesto por tres activos ($A$, $B$, $C$), definimos:

- Rendimientos esperados:
  $$
  R_A = 10\%, \quad R_B = 8\%, \quad R_C = 12\%
  $$

- Riesgos (desviaciones estándar):
  $$
  \sigma_A = 15\%, \quad \sigma_B = 10\%, \quad \sigma_C = 20\%
  $$

- Matriz de correlación:
  $$
  \text{Correlación} =
  \begin{bmatrix}
  1 & 0.5 & 0.4 \\
  0.5 & 1 & 0.3 \\
  0.4 & 0.3 & 1
  \end{bmatrix}
  $$

El objetivo es minimizar el riesgo del portafolio para un rendimiento esperado del $9\%$.

## Paso 2: Fórmula del Rendimiento del Portafolio

El rendimiento esperado del portafolio se calcula como:

$$
R_p = w_A R_A + w_B R_B + w_C R_C
$$

Donde:
- $w_A, w_B, w_C$ son las proporciones de los activos en el portafolio.
- $w_A + w_B + w_C = 1$.

## Paso 3: Fórmula del Riesgo del Portafolio

El riesgo del portafolio (desviación estándar) se calcula como:

$$
\sigma_p^2 = w^T \Sigma w
$$

Donde:
- $w = \begin{bmatrix} w_A \\ w_B \\ w_C \end{bmatrix}$ es el vector de pesos,
- $\Sigma$ es la matriz de covarianza, obtenida a partir de las desviaciones estándar y la matriz de correlación.

Matriz de covarianza:

$$
\Sigma =
\begin{bmatrix}
\sigma_A^2 & \rho_{AB} \sigma_A \sigma_B & \rho_{AC} \sigma_A \sigma_C \\
\rho_{AB} \sigma_A \sigma_B & \sigma_B^2 & \rho_{BC} \sigma_B \sigma_C \\
\rho_{AC} \sigma_A \sigma_C & \rho_{BC} \sigma_B \sigma_C & \sigma_C^2
\end{bmatrix}
$$

## Paso 4: Restricciones de Optimización

1. $R_p = 9\%$: El rendimiento objetivo.
2. $w_A + w_B + w_C = 1$: Suma de las proporciones.
3. $w_A, w_B, w_C \geq 0$: Pesos no negativos.

## Paso 5: Resolución

El problema se resuelve minimizando $\sigma_p^2$ sujeto a las restricciones, utilizando métodos como programación cuadrática.

## Paso 6: Frontera Eficiente

Para construir la frontera eficiente, se calculan los riesgos mínimos para diferentes valores de rendimiento objetivo, trazando una curva que muestra la relación riesgo-rendimiento.



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize

# Datos del problema
rendimientos = np.array([0.10, 0.08, 0.12])  # Rendimientos esperados de A, B, C
desviaciones = np.array([0.15, 0.10, 0.20])  # Desviaciones estándar de A, B, C
correlacion = np.array([
    [1.0, 0.5, 0.4],
    [0.5, 1.0, 0.3],
    [0.4, 0.3, 1.0]
])  # Matriz de correlación

# Matriz de covarianza
covarianza = np.outer(desviaciones, desviaciones) * correlacion

# Función objetivo: minimizar el riesgo (varianza)
def riesgo_portafolio(pesos):
    return np.dot(pesos.T, np.dot(covarianza, pesos))  # Varianza del portafolio

# Restricciones
def rendimiento_objetivo(pesos):
    return np.dot(pesos, rendimientos) - 0.09  # Rendimiento objetivo del 9%

def suma_pesos(pesos):
    return np.sum(pesos) - 1  # Pesos suman 1

# Restricciones y límites
restricciones = ({'type': 'eq', 'fun': suma_pesos},
                 {'type': 'eq', 'fun': rendimiento_objetivo})
limites = [(0, 1) for _ in range(3)]  # Pesos no negativos

# Optimización
pesos_iniciales = np.array([1/3, 1/3, 1/3])  # Pesos iniciales iguales
resultado = minimize(riesgo_portafolio, pesos_iniciales, constraints=restricciones, bounds=limites)

pesos_optimos = resultado.x
riesgo_optimo = np.sqrt(resultado.fun)  # Riesgo mínimo (desviación estándar)

# Generar la frontera eficiente
rendimientos_objetivo = np.linspace(0.08, 0.12, 100)
riesgos = []

for r_obj in rendimientos_objetivo:
    restricciones = ({'type': 'eq', 'fun': suma_pesos},
                     {'type': 'eq', 'fun': lambda pesos: np.dot(pesos, rendimientos) - r_obj})
    resultado = minimize(riesgo_portafolio, pesos_iniciales, constraints=restricciones, bounds=limites)
    riesgos.append(np.sqrt(resultado.fun))

# Gráfica de la frontera eficiente
plt.figure(figsize=(12, 6))
plt.plot(riesgos, rendimientos_objetivo, label='Frontera Eficiente', color='blue')
plt.scatter(riesgo_optimo, 0.09, color='red', label='Portafolio Óptimo (Rend. 9%)', zorder=5)
plt.title('Frontera Eficiente del Portafolio', fontsize=16)
plt.xlabel('Riesgo (Desviación Estándar)', fontsize=14)
plt.ylabel('Rendimiento Esperado', fontsize=14)
plt.legend(fontsize=12)
plt.grid(True)
plt.show()

pesos_optimos, riesgo_optimo